In [14]:
%load_ext autoreload
%autoreload 2

In [15]:
import os
import json
from pprint import pprint
from typing import Dict, Any
from google import genai
from dotenv import load_dotenv
from cfg_builder import *
from prompts import *

In [2]:
load_dotenv()
client = genai.Client(api_key=os.getenv('GOOGLE_API_KEY'))

### Retrieve codebase

In [3]:
codebase = json.load(open("../data/sample_codebase.json"))
list(codebase.keys())

['/project/app/main.py',
 '/project/app/utils/data_loader.py',
 '/project/app/utils/math_ops.py',
 '/project/app/services/processor.py',
 '/project/app/services/report.py',
 '/project/app/tests/test_math_ops.py',
 '/project/app/tests/test_processor.py']

### CFG Builder

In [5]:
generated_cfg = build_cfg(codebase)
generated_cfg

{'project': 'generated_project',
 'files': {'/project/app/main.py': {'functions': {'main': {'entry_node': 'main.py.main.entry_1',
     'exit_node': 'main.py.main.exit_2',
     'nodes': [{'id': 'main.py.main.entry_1',
       'type': 'ENTRY',
       'text': '',
       'line': None,
       'indent': 0,
       'metadata': {}},
      {'id': 'main.py.main.assign_3',
       'type': 'ASSIGN',
       'text': "numbers = load_numbers('data/input.txt')",
       'line': 6,
       'indent': 0,
       'metadata': {}},
      {'id': 'main.py.main.call_4',
       'type': 'CALL',
       'text': "load_numbers('data/input.txt')",
       'line': 6,
       'indent': 0,
       'metadata': {'called_function': 'load_numbers'}},
      {'id': 'main.py.main.expr_5',
       'type': 'EXPR',
       'text': "print('Loaded numbers:', numbers)",
       'line': 7,
       'indent': 0,
       'metadata': {}},
      {'id': 'main.py.main.call_6',
       'type': 'CALL',
       'text': "print('Loaded numbers:', numbers)",
    

In [9]:
out_path = "../data/generated_cfg.json"
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(generated_cfg, f, indent=2)
print("Saved CFG to", out_path)

# Print a small preview (first file)
for i in range(len(generated_cfg["files"])):
    first = list(generated_cfg["files"].keys())[i]
    print("Preview file:", first)
    print("Functions:", list(generated_cfg["files"][first]["functions"].keys()))

Saved CFG to ../data/generated_cfg.json
Preview file: /project/app/main.py
Functions: ['main', '<module>']
Preview file: /project/app/utils/data_loader.py
Functions: ['load_numbers', '<module>']
Preview file: /project/app/utils/math_ops.py
Functions: ['compute_average', 'compute_factorial', '<module>']
Preview file: /project/app/services/processor.py
Functions: ['NumberProcessor.__init__', 'NumberProcessor.process', 'NumberProcessor.unstable_sort', '<module>']
Preview file: /project/app/services/report.py
Functions: ['generate_report', '<module>']
Preview file: /project/app/tests/test_math_ops.py
Functions: ['test_compute_average_basic', 'test_compute_average_empty', 'test_factorial_zero', '<module>']
Preview file: /project/app/tests/test_processor.py
Functions: ['test_processor_flow', 'test_sort', '<module>']


### LLM Call

In [17]:
debug_prompt = get_debug_prompt()
input_sample = get_sample_input()
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=[debug_prompt + '\nINPUT:\n' + input_sample],
    config={
        # "response_mime_type": "application/json",
        # 'response_schema': 
    }
)

In [18]:
pprint(response.text)

('1. Root Cause Summary\n'
 '\n'
 "The true root cause is an incorrect modification to the loop's iteration "
 'range in `analyze_numbers`. The change from `range(len(nums))` to '
 '`range(len(nums) - 1)` prevents the loop from iterating over all intended '
 'indices. This has two critical consequences: first, if `nums` is an empty '
 'list (`len(nums)` is 0), `range(-1)` is called, which raises a `ValueError` '
 '(reported as `IndexError: range() step cannot be negative` in the logs). '
 'Second, for non-empty lists, the loop terminates prematurely, missing the '
 'last potential iteration (index `len(nums) - 1`), which can lead to the '
 'function incorrectly returning `-1` instead of a valid index that would have '
 'satisfied the condition `avg > threshold`.\n'
 '\n'
 '2. Evidence from CFG + Trace\n'
 '\n'
 '*   **CFG Node a2**: The relevant code snippet `for i in range(len(nums) - '
 '1):` is located at CFG node `a2` in `project/services/analyzer.py`. This is '
 'the loop header w

In [ ]:
# if response.text is not None:
#     json_output = json.loads(response.text)